In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Autoencoder Applications\n",
    "## Этап 5: Применения автоэнкодера\n",
    "\n",
    "В этом ноутбуке:\n",
    "1. **Детекция аномалий** - находим странные изображения\n",
    "2. **Визуализация латентного пространства** - смотрим как модель \"понимает\" картинки\n",
    "3. **Деноизинг** - очищаем зашумленные изображения\n",
    "4. **Генерация** - создаем новые изображения (VAE)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "from torch.utils.data import DataLoader\n",
    "from torchvision import datasets, transforms\n",
    "import sys\n",
    "import os\n",
    "\n",
    "sys.path.append('..')\n",
    "\n",
    "from models.autoencoder import SimpleAutoencoder, VAE\n",
    "from utils.visualization import (\n",
    "    visualize_reconstructions,\n",
    "    visualize_latent_space,\n",
    "    visualize_generated_images,\n",
    "    visualize_denoising\n",
    ")\n",
    "from utils.anomaly_detection import (\n",
    "    calculate_reconstruction_error,\n",
    "    detect_anomalies,\n",
    "    visualize_anomalies,\n",
    "    plot_error_distribution\n",
    ")\n",
    "\n",
    "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n",
    "print(f'Device: {device}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Подготовка данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Параметры\n",
    "DATA_DIR = '../data/animal_faces'\n",
    "BATCH_SIZE = 32\n",
    "LATENT_DIM = 128\n",
    "\n",
    "# Трансформации\n",
    "transform = transforms.Compose([\n",
    "    transforms.Resize((224, 224)),\n",
    "    transforms.ToTensor()\n",
    "])\n",
    "\n",
    "# Загрузка данных\n",
    "val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'val'), transform=transform)\n",
    "val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)\n",
    "\n",
    "print(f\"Val dataset: {len(val_dataset)} images\")\n",
    "print(f\"Classes: {val_dataset.classes}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Загрузка обученных моделей"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Загружаем Simple Autoencoder\n",
    "ae = SimpleAutoencoder(latent_dim=LATENT_DIM).to(device)\n",
    "ae.load_state_dict(torch.load('../checkpoints/autoencoder_best.pth', map_location=device))\n",
    "ae.eval()\n",
    "print(\"✓ Loaded Simple Autoencoder\")\n",
    "\n",
    "# Загружаем VAE\n",
    "vae = VAE(latent_dim=LATENT_DIM).to(device)\n",
    "vae.load_state_dict(torch.load('../checkpoints/vae_best.pth', map_location=device))\n",
    "vae.eval()\n",
    "print(\"✓ Loaded VAE\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Реконструкция изображений\n",
    "### Проверяем как модель восстанавливает изображения"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Simple Autoencoder Reconstructions:\")\n",
    "visualize_reconstructions(\n",
    "    ae, val_loader, device, \n",
    "    num_images=8,\n",
    "    save_path='../results/autoencoder/ae_reconstructions.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"VAE Reconstructions:\")\n",
    "visualize_reconstructions(\n",
    "    vae, val_loader, device, \n",
    "    num_images=8,\n",
    "    save_path='../results/autoencoder/vae_reconstructions.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Детекция аномалий\n",
    "### Находим \"странные\" изображения с большой ошибкой реконструкции"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Calculating reconstruction errors...\")\n",
    "errors, images, labels = calculate_reconstruction_error(ae, val_loader, device)\n",
    "print(f\"✓ Calculated errors for {len(errors)} images\")\n",
    "\n",
    "# Находим аномалии\n",
    "anomalies, threshold = detect_anomalies(errors, threshold_percentile=95)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Распределение ошибок\n",
    "plot_error_distribution(\n",
    "    errors, labels,\n",
    "    save_path='../results/autoencoder/error_distribution.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Визуализация нормальных и аномальных изображений\n",
    "visualize_anomalies(\n",
    "    errors, images, labels,\n",
    "    num_normal=4,\n",
    "    num_anomalies=4,\n",
    "    save_path='../results/autoencoder/anomaly_detection.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Визуализация латентного пространства\n",
    "### Смотрим как модель \"понимает\" разные классы"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Visualizing latent space...\")\n",
    "visualize_latent_space(\n",
    "    ae, val_loader, device,\n",
    "    save_path='../results/autoencoder/latent_space.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Деноизинг (Очистка от шума)\n",
    "### Автоэнкодер может очищать зашумленные изображения"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Denoising with Simple Autoencoder:\")\n",
    "visualize_denoising(\n",
    "    ae, val_loader, device,\n",
    "    noise_level=0.3,\n",
    "    num_images=8,\n",
    "    save_path='../results/autoencoder/denoising_ae.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Denoising with VAE:\")\n",
    "visualize_denoising(\n",
    "    vae, val_loader, device,\n",
    "    noise_level=0.3,\n",
    "    num_images=8,\n",
    "    save_path='../results/autoencoder/denoising_vae.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Генерация новых изображений (VAE)\n",
    "### VAE может создавать новые изображения, которых не было в датасете"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Generating new images with VAE...\")\n",
    "visualize_generated_images(\n",
    "    vae, device,\n",
    "    num_images=16,\n",
    "    save_path='../results/autoencoder/generated_images.png'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Итоговая сводка"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n\" + \"=\"*60)\n",
    "print(\"AUTOENCODER APPLICATIONS SUMMARY\")\n",
    "print(\"=\"*60)\n",
    "print(\"\\n✓ Completed applications:\")\n",
    "print(\"  1. Image Reconstruction\")\n",
    "print(\"  2. Anomaly Detection\")\n",
    "print(\"  3. Latent Space Visualization\")\n",
    "print(\"  4. Denoising\")\n",
    "print(\"  5. Image Generation (VAE)\")\n",
    "print(\"\\n✓ Results saved to: ../results/autoencoder/\")\n",
    "print(\"\\n\" + \"=\"*60)\n",
    "\n",
    "# Статистика аномалий\n",
    "print(\"\\nAnomaly Detection Statistics:\")\n",
    "print(f\"  Total images: {len(errors)}\")\n",
    "print(f\"  Anomalies found: {np.sum(anomalies)} ({100*np.sum(anomalies)/len(errors):.1f}%)\")\n",
    "print(f\"  Threshold: {threshold:.4f}\")\n",
    "print(f\"  Mean error: {np.mean(errors):.4f}\")\n",
    "print(f\"  Max error: {np.max(errors):.4f}\")\n",
    "print(f\"  Min error: {np.min(errors):.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Выводы\n",
    "\n",
    "Мы протестировали автоэнкодеры и показали их применения:\n",
    "\n",
    "1. **Детекция аномалий**: Нашли изображения, которые плохо реконструируются (возможно из-за необычного ракурса, освещения или качества)\n",
    "\n",
    "2. **Визуализация латентного пространства**: Увидели как модель группирует разные классы животных\n",
    "\n",
    "3. **Деноизинг**: Автоэнкодер может очищать зашумленные изображения\n",
    "\n",
    "4. **Генерация (VAE)**: VAE может создавать новые изображения животных\n",
    "\n",
    "**Простыми словами:**\n",
    "- Автоэнкодер сжимает изображение в маленький вектор (как архив)\n",
    "- Потом пытается восстановить обратно\n",
    "- Если не получается хорошо восстановить - значит изображение \"странное\"\n",
    "- VAE может даже генерировать новые картинки!"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}